# prologue

### set up notebook and load package

In [1]:
# load what we need
import numpy as np
import CHIRPS.datasets as ds
import CHIRPS.datasets_proprietary as dsp
import CHIRPS.reproducible as rp
import multiprocessing as mp

# data management

### datasets

Any dataset can be turned into a container by invoking the constructor found in the module structures.py, the only mandatory parameters are an object that can be input to pandas.DataFrame and a class column name. Your object needs either its own column names, or these should be passed as a list to the var_names parameter.

Several datasets are available as pre-prepared containers that hold the data and some meta-data that is used in the algorithm. If these pre-packaged sets have more than 10,000 rows then we've included some downsampled versions. You can see what there is by doing:

In [2]:
# ignore the builtins, np, pd, urllib and cfg. The rest are dataset constructors that will load specific datasets.
if False:
    dir(ds)

You can see more information about each dataset with something like this for the adult dataset:

In [3]:
if False:
    print(ds.adult().spiel)

An example of one dataset is given below. Note that the random_state propagates through other functions via the meta_data and is easily updated to allow alternative runs:

In [4]:
if False:
    cardio = ds.cardio(random_state=123, project_dir=project_dir)
    meta_data = cardio.get_meta()
    meta_data['random_state']

The list of datasets used in "CHIRPS: Explaining Random Forest Classification" is:

In [5]:
if False:
    datasets = [
            ds.adult,
            ds.bankmark,
            ds.car,
            ds.cardio,
            ds.credit,
            ds.german,
            ds.lending_tiny_samp,
            ds.nursery,
            ds.rcdv,
           ]

The list of datasets used in "Explaining Multi-class AdaBoost Classification in Medical Applications" is:

In [6]:
if False:
    datasets = [
                ds.breast,
                ds.cardio,
                ds.diaretino,
                ds.heart,
                ds.mhtech14,
                ds.mh2tech16,
                ds.readmit,
                ds.thyroid,
                dsp.usoc2,
               ]

### Standardising train-test splitting across environments
To compare with methods in different environments, as we did in "CHIRPS: Explaining Random Forest Classification," you can maintain the same dataset splits. The train test data is split with the one-time random_state_splits and the splits are saved to csv in the project folders. This way you can do same model with different data splits, or same data splits with different model. This is what's happening behind the scenes:

In [7]:
##### do not run this here
# train_index, test_index = mydata.get_tt_split_idx(random_state=random_state_splits)
# tt = mydata.tt_split(train_index, test_index)

All you need here is the following, to take the same train_index, test_index numbers to an external .csv file

# Experimental Runs
From here on, everything is automated behind the scenes to perform runs across several datasets and comparing different algorithms with CHIRPS. If you want to run CHIRPS and control the parameters and view results and performance directly, look at the CHIRPS Examples Notebook.

In [8]:
# do the model tuning from scratch?
override_tuning = False

# Optional Memory and Computation Cost Management
# CHIRPS is time economical but memory intensive to compute for lots of instances at once.
forest_walk_async=True
explanation_async=True
n_cores = mp.cpu_count()-2

# How many instances from the test set do you want to explain?
# A number bigger than the test set will be interpreted as 'all'
n_instances = 1000
start_instance = 0 # here can opt to start at a specific instance, diagnostic if something crashes

# CHOOSE ONE
# model = 'RandomForest'
# model = 'AdaBoost1' # SAMME
# model = 'AdaBoost2' # SAMME.R
model = 'GBM'

# CHOOSE ONE OR MORE
do_CHIRPS = False
do_Anchors = False
do_dfrgTrs = True
do_lore = False

# list the dataset constructors you want to include
datasets = [
            ds.adult,
#             ds.bankmark,
#             ds.car,
#             ds.cardio,
#             ds.credit,
#             ds.german,
#             ds.lending_tiny_samp,
#             ds.nursery,
            ds.rcdv,
           ]

# location to save results
project_dir = '/datadisk/whiteboxing/2020GBM_copy'
# project_dir = 'V:\\whiteboxing\\2020' # defaults to a directory "whiteboxing" in the working directory
# project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\2020'

# set the random_state for various tasks. This is not the same as random.seed(), not system wide.
random_state_splits = 123 # change this if you want to try different splits of the data into test / train
random_state_rf = 123 # change this if you want to try with different forest construction
random_state_exp = 123 # change this if you want to try with different runs of the explainer algorithm (affects bootstrap eval)

# To standardise train-test splitting across environments - writes external files
rp.export_data_splits(datasets=datasets, project_dir=project_dir, random_state_splits=random_state_splits)

# How much messaging to print to the screen?
verbose = True

# CHIRPS default parameters - see papers for details
merging_bootstraps = 20 # how many training bootstraps to test improvement in growing rule?
pruning_bootstraps = 20 # how many training bootstraps to test deterioration in pruning rule?
delta = 0.2 # pruning deterioration tolerance paramater

# this is here if you want to pass parameters to the methods
def benchmark_wrapper(do_CHIRPS=do_CHIRPS, do_Anchors=do_Anchors, do_dfrgTrs=do_dfrgTrs, do_lore=do_lore, **control):

    if do_CHIRPS:
        rp.do_benchmarking(benchmark_items, verbose, **control)
    
    if do_Anchors:
        control.update({'method' : 'Anchors'})
        rp.do_benchmarking(benchmark_items, verbose, **control)
    if do_dfrgTrs:
        control.update({'method' : 'defragTrees',
                        'Kmax' : 10, 'restart' : 10, 'maxitr' : 100})
        rp.do_benchmarking(benchmark_items, verbose, **control)

    if do_lore:
        control.update({'method' : 'lore',
                        'max_popsize' : 1000})
        rp.do_benchmarking(benchmark_items, verbose, **control)

# this is here to pass parameters to the model training and further parameters to CHIRPS
tuning = {'override' : override_tuning}
if model == 'RandomForest':
    tuning.update({'grid' : {'n_estimators': [(i + 1) * 200 for i in range(8)],
                            'max_depth' : [32]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=random_state_rf,
                                           random_state_splits=random_state_splits,
                                           do_raw=True, do_discretise=do_Anchors,
                                           start_instance=start_instance,
                                           verbose=verbose, n_cores=n_cores)

    kwargs = {'support_paths' : 0.2, 'alpha_paths' : 0.5, 'disc_path_bins' : 8,
             'score_func' : 1, 'weighting' : 'kldiv',
             'merging_bootstraps' : merging_bootstraps,
             'pruning_bootstraps' : pruning_bootstraps,
             'precis_threshold' : 0.99,
             'delta' : delta}
 
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'explanation_async' : explanation_async,
                'n_cores' : n_cores}
    
    benchmark_wrapper(do_CHIRPS, do_Anchors, do_dfrgTrs, do_lore, **control)
    
elif model == "AdaBoost1":
    algo = 'SAMME'
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)],
                             'algorithm': [algo]}}) 
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                       random_state=random_state_rf,
                                       random_state_splits=random_state_splits,
                                       do_raw=(do_CHIRPS or do_dfrgTrs or do_lore), do_discretise=do_Anchors,
                                       start_instance=start_instance,
                                       verbose=verbose, n_cores=n_cores)
    
    kwargs = {'paths_lengths_threshold' : 5,
             'support_paths' : 0.01, 'alpha_paths' : 0.0,
             'disc_path_bins' : 8, 'disc_path_eqcounts' : False,
             'score_func' : 1, 'weighting' : 'chisq',
             'merging_bootstraps' : merging_bootstraps,
             'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
 
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'explanation_async' : explanation_async,
                'n_cores' : n_cores}
    
    benchmark_wrapper(do_CHIRPS, do_Anchors, do_dfrgTrs, do_lore, **control)
    
elif model == 'AdaBoost2':
    algo = 'SAMME.R'
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)],
                             'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                       random_state=random_state_rf,
                                       random_state_splits=random_state_splits,
                                       do_raw=(do_CHIRPS or do_dfrgTrs or do_lore), do_discretise=do_Anchors,
                                       start_instance=start_instance,
                                       verbose=verbose, n_cores=n_cores)
    
    kwargs = {'paths_lengths_threshold' : 5,
                 'support_paths' : 0.005, 'alpha_paths' : 0.0,
                 'disc_path_bins' : 8, 'disc_path_eqcounts' : False,
                 'score_func' : 1, 'weighting' : 'chisq',
                 'merging_bootstraps' : merging_bootstraps,
                 'pruning_bootstraps' : pruning_bootstraps,
                 'which_trees' : 'majority',
                 'delta' : delta}
    
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'explanation_async' : explanation_async,
                'n_cores' : n_cores}
    
    benchmark_wrapper(do_CHIRPS, do_Anchors, do_dfrgTrs, do_lore, **control)
    
else: # GBM - not fully implemented yet
    tuning.update({'grid' : {'subsample' : [0.5],
                            'n_estimators': [i * 200 for i in range(1, 9)],
                            'max_depth' : [i for i in range(1, 5)],
                            'learning_rate': np.full(4, 10.0)**[i for i in range(-3, 1)]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                       random_state=random_state_rf,
                                       random_state_splits=random_state_splits,
                                       do_raw=(do_CHIRPS or do_dfrgTrs or do_lore), do_discretise=do_Anchors,
                                       start_instance=start_instance,
                                       verbose=verbose, n_cores=n_cores)
    
    kwargs = {'paths_lengths_threshold' : 5,
             'support_paths' : 0.05, 'alpha_paths' : 0.0,
             'disc_path_bins' : 4, 'disc_path_eqcounts' : True,
             'score_func' : 1, 'weighting' : 'kldiv',
             'merging_bootstraps' : merging_bootstraps,
             'pruning_bootstraps' : pruning_bootstraps,
             'which_trees' : 'targetclass',
             'delta' : 0.2}
    
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'explanation_async' : explanation_async,
                'n_cores' : n_cores}
    
    benchmark_wrapper(do_CHIRPS, do_Anchors, do_dfrgTrs, do_lore, **control)

print('finished')

/home/julianhatwell/anaconda3/envs/B3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Exported train-test data for 2 datasets.
Preprocessing adult data and model for adult with random state = 123
Split data into main train-test and build forest


/home/julianhatwell/anaconda3/envs/B3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Train main model
using previous tuning parameters
Best OOB Accuracy Estimate during tuning: 0.8707
Best parameters:{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 1600, 'subsample': 0.5, 'random_state': 123}


Preprocessing rcdv data and model for rcdv with random state = 123
Split data into main train-test and build forest
Train main model
using previous tuning parameters
Best OOB Accuracy Estimate during tuning: 0.6802
Best parameters:{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.5, 'random_state': 123}


Running defragTrees

[Seed 123] TrainingError = 0.16, K = 5
[Seed 124] TrainingError = 0.17, K = 6
[Seed 125] TrainingError = 0.24, K = 2
[Seed 126] TrainingError = 0.24, K = 4
[Seed 127] TrainingError = 0.24, K = 3
[Seed 128] TrainingError = 0.17, K = 6
[Seed 129] TrainingError = 0.22, K = 5
[Seed 130] TrainingError = 0.16, K = 6
[Seed 131] TrainingError = 0.18, K = 4
[Seed 132] TrainingError = 0.17, K = 5
Optimal Model >> Seed 130, TrainingEr

380: Working on defragTrees for instance 4616
390: Working on defragTrees for instance 9345
400: Working on defragTrees for instance 1559
410: Working on defragTrees for instance 15685
420: Working on defragTrees for instance 14499
430: Working on defragTrees for instance 7983
440: Working on defragTrees for instance 8643
450: Working on defragTrees for instance 2430
460: Working on defragTrees for instance 2379
470: Working on defragTrees for instance 6038
480: Working on defragTrees for instance 2223
490: Working on defragTrees for instance 14255
500: Working on defragTrees for instance 10038
510: Working on defragTrees for instance 9366
520: Working on defragTrees for instance 5183
530: Working on defragTrees for instance 15075
540: Working on defragTrees for instance 18111
550: Working on defragTrees for instance 6024
560: Working on defragTrees for instance 18654
570: Working on defragTrees for instance 4747
580: Working on defragTrees for instance 18305
590: Working on defragTree